In [1]:
import graphlab as gl
import scipy as sp
import numpy as np
import xgboost as xgb

from HAMMER import train_xgboost, xgboost_predict

[WARNING] Unable to write current GraphLab Create license to /home/alvas/.graphlab/config. Ensure that this user account has write permission to /home/alvas/.graphlab/config to save the license for offline use.
[INFO] This non-commercial license of GraphLab Create is assigned to liling@coli.uni-saarland.de and will expire on October 11, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-30374 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454464092.log
[INFO] GraphLab Server Version: 1.8.1
[WARNING] Unable to create session in specified location: '/home/alvas/.graphlab/artifacts'. Using: '/var/tmp/graphlab-alvas/30374/tmp_session_2eb4e4b3-04bd-4751-9974-c79bb5133d3b'


In [2]:
sts_train = gl.SFrame('../sts2016_train.stasis/')
sts_test = gl.SFrame('../sts2016_test.stasis/')
sts_train = sts_train.dropna(columns=['Score'])

In [3]:
feat1 = ['prop_harmonic', 'DLS_compose_ppmi', 'DLS_compose_cbow']
feat2 = ['glove_cosine', 'prop_harmonic', 'DLS_compose_ppmi', 'DLS_compose_cbow', 'REVAL', 'BEER']
m1 = gl.boosted_trees_regression.create(sts_train, target='Score', features=feat1, validation_set=None)
m2 = gl.boosted_trees_regression.create(sts_train, target='Score', features=feat2, validation_set=None)



PROGRESS: Boosted trees regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 13597
PROGRESS: Number of features          : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 1.029581     | 4.26247            | 2.12918       |
PROGRESS: | 2         | 1.036718     | 4.04426            | 1.65681       |
PROGRESS: | 3         | 1.045100     | 3.99651            | 1.36361       |
PROGRESS: | 4         | 1.052687     | 3.99096            | 1.18997       |
PROGRESS: | 5         | 1.060311     | 3.99484            | 1.09257       |
PROGRESS: | 6         | 1.068820     | 4.00021            | 1.03826       |
PROGRESS: | 7         | 1.077671     | 4.03883            | 1.0079        |
PROGRES

In [4]:
train_valid = sts_train['glove_cosine', 'prop_harmonic', 'DLS_compose_ppmi', 'DLS_compose_cbow', 'REVAL', 'BEER', 'Score'].to_dataframe()

In [5]:
gbm = train_xgboost(train_valid, feat2, num_boost_round=5000, eta=0.02, max_depth=8, test_size=0.2)

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-rmse:0.896152	eval-rmse:0.901518
[1]	train-rmse:0.880577	eval-rmse:0.886048
[2]	train-rmse:0.865539	eval-rmse:0.871229
[3]	train-rmse:0.850636	eval-rmse:0.856462
[4]	train-rmse:0.836006	eval-rmse:0.842031
[5]	train-rmse:0.821759	eval-rmse:0.828008
[6]	train-rmse:0.808060	eval-rmse:0.814521
[7]	train-rmse:0.794441	eval-rmse:0.801071
[8]	train-rmse:0.781371	eval-rmse:0.788285
[9]	train-rmse:0.768570	eval-rmse:0.775681
[10]	train-rmse:0.756081	eval-rmse:0.763567
[11]	train-rmse:0.743806	eval-rmse:0.751607
[12]	train-rmse:0.731799	eval-rmse:0.739883
[13]	train-rmse:0.719854	eval-rmse:0.728134
[14]	train-rmse:0.708187	eval-rmse:0.716678
[15]	train-rmse:0.696814	eval-rmse:0.705497
[16]	train-rmse:0.685873	eval-rmse:0.694909
[17]	train-rmse:0.675261	eval-rmse:0.684585
[18]	train-rmse:0.664602	eval-rmse:0.674208
[19]	train-rmse:0.654332	eval-rmse:0.664331
[20]	train-rmse:0.644129	eval-rmse:0.654446
[21]	train-rmse:0.634107	e

Validating
RMSE: 0.988670


[644]	train-rmse:0.194084	eval-rmse:0.324593
[645]	train-rmse:0.193966	eval-rmse:0.324593
Stopping. Best iteration:
[545]	train-rmse:0.203218	eval-rmse:0.324563



In [6]:
predictions = xgboost_predict(gbm, sts_test[feat2].to_dataframe(), feat2)


In [7]:
import os, io

def output_to_file(team_name, run_name, target):
    test_dir = '../sts2016-english-v1.1/'
    filenames = [test_dir+i for i in os.listdir(test_dir) if i.endswith('ascii')]
    for infile in filenames:
        domain = infile.rpartition('.')[0].rpartition('.')[2]
        outfile = 'STS2016.OUTPUT.'+team_name+'.'+run_name+'.'+domain+'.txt'
        with io.open(infile, 'r') as fin, io.open(outfile, 'w') as fout:
            domain_data = sts_test.filter_by(domain, column_name='Domain')
            for line, row in zip(fin, domain_data):
                #line = line.split('\t')
                #s1, s2 = line[0], line[1]
                fout.write(unicode(row[target])+'\n')

sts_test.add_column(gl.SArray(m1.predict(sts_test)), name='pred_m1')
sts_test.add_column(gl.SArray(m2.predict(sts_test)), name='pred_m2')                
sts_test.add_column(gl.SArray(predictions), name='pred_xgboost')
output_to_file('wolvesaar', 'DLS-replica', 'pred_m1')
output_to_file('wolvesaar', 'lotsa-embeddings', 'pred_m2')
output_to_file('wolvesaar', 'xgboost', 'pred_xgboost')





